# Upload Data and Import Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from zipfile import ZipFile 
# specifying the zip file name 
file_name = "/content/drive/My Drive/Zindi/Competitions/ZINDI_CGIAR/data.zip"
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zi: 
    # extracting all the files 
    print('Extracting all the files now...') 
    zi.extractall() 
    print('Done!')

Extracting all the files now...
Done!


In [ ]:
!cp "/content/drive/My Drive/Zindi/Competitions/ZINDI_CGIAR/Train.csv" . 
!cp "/content/drive/My Drive/Zindi/Competitions/ZINDI_CGIAR/SampleSubmission.csv" .

In [ ]:
!git clone https://github.com/clovaai/rexnet.git

Cloning into 'rexnet'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 16 (delta 5), reused 9 (delta 2), pack-reused 0
Unpacking objects: 100% (16/16), done.


In [ ]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp36-none-any.whl size=16031 sha256=2f786292a6144e3da3ac47542178706edc14b815e29a40f0f2c4b446a3bf485c
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch


In [ ]:
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torchvision import transforms
import torch
import torchvision
import torchvision.models as models
from efficientnet_pytorch import EfficientNet
from torch.optim.lr_scheduler import MultiStepLR
from torch.optim.lr_scheduler import OneCycleLR
import pandas as pd 
import numpy as np
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from tqdm import tqdm_notebook as tqdm 
from sklearn.model_selection import train_test_split

In [ ]:
import random
import numpy as np
SEED_VAL  = 20212020
# Set the seed value all over the place to make this reproducible.
def seed_all(SEED):
  random.seed(SEED_VAL)
  np.random.seed(SEED_VAL)
  torch.manual_seed(SEED_VAL)
  torch.cuda.manual_seed_all(SEED_VAL)
  os.environ['PYTHONHASHSEED'] = str(SEED_VAL)
  torch.backends.cudnn.deterministic = True

# Preparing Dataset 

In [ ]:
import cv2
def default_image_loader(path):
    return Image.open(path).convert('RGB')

def cv_reader(path):
  img = cv2.imread(path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

class ImageDataset(Dataset):
    def __init__(self, data_path, df, transform=None,mode='train'):
        self.df = df
        self.loader = cv_reader
        self.transform = transform
        self.dir = data_path
        self.mode = mode

    def __getitem__(self, index):
        image_name = self.df.image_name[index]
        image = self.loader(os.path.join(self.dir, image_name+'.jpeg'))
        if self.transform is not None:
          image = self.transform(image=image)
          image=image["image"]
        

        if self.mode == 'train':
            label = self.df.target[index]
            return {'image' : torch.tensor(image,dtype=torch.float), 
                'label' : torch.tensor(label,dtype = torch.float) }
            
        return {'image' : torch.tensor(image,dtype=torch.float), 
}            
        
    
    def __len__(self):
        return self.df.shape[0]

In [ ]:
class Net(nn.Module):
    def __init__(self,name):
        super(Net, self).__init__()
        self.name  = name
        if name == 'b0':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b0')
          self.arch._fc = nn.Linear(in_features=1280, out_features=1, bias=True)
        elif name == 'b1':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b1')
          self.arch._fc = nn.Linear(in_features=1280, out_features=1, bias=True)
        elif name == 'b2':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b2')
          self.arch._fc = nn.Linear(in_features=1408, out_features=1, bias=True)
        elif name =='b3':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b3')
          self.arch._fc = nn.Linear(in_features=1536, out_features=1, bias=True)
        elif name =='b4':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b4')
          self.arch._fc = nn.Linear(in_features=1792, out_features=1, bias=True)
        elif name =='b5':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b5')
          self.arch._fc = nn.Linear(in_features=2048, out_features=1, bias=True)
        elif name =='b6':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b6')
          self.arch._fc = nn.Linear(in_features=2304, out_features=1, bias=True)
        elif name =='b7':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b7')
          self.arch._fc = nn.Linear(in_features=2560, out_features=1, bias=True)
        elif name == 'densenet121':
          self.arch = models.densenet121(pretrained=True)
          num_ftrs = self.arch.classifier.in_features
          self.arch.classifier = nn.Linear(num_ftrs,1,bias=True)
        elif name == 'densenet169':
          self.arch = models.densenet169(pretrained=True)
          num_ftrs = self.arch.classifier.in_features
          self.arch.classifier = nn.Linear(num_ftrs,1,bias=True)
        elif name == 'densenet201':
          self.arch = models.densenet201(pretrained=True)
          num_ftrs = self.arch.classifier.in_features
          self.arch.classifier = nn.Linear(num_ftrs,1,bias=True)
        elif name == 'resnet50':
          self.arch = models.resnet50(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,1,bias=True)
        elif name == 'resnet101':
          self.arch = models.resnet101(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,1,bias=True)
        elif name == 'resnet152':
          self.arch = models.resnet152(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,1,bias=True)
        elif name == 'resnet34':
          self.arch = models.resnet34(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,1,bias=True)
        elif name == 'resnext101':
          self.arch = models.resnext101_32x8d(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,1,bias=True)
        elif name == 'resnext50':
          self.arch = models.resnext50_32x4d(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          
          self.arch.fc = nn.Linear(num_ftrs,1,bias=True)
        elif name =='rexnetv1':
            model = rexnetv1.ReXNetV1(width_mult=1.0)
            model.output.conv2D = nn.Conv2d(1280, 1, kernel_size=(1, 1), stride=(1, 1))

          
    def forward(self, x):
        """
        """
        x = self.arch(x)
        return x

In [ ]:
class AverageMeter():
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def loss_fn(outputs,targets):
  criterion = nn.MSELoss()
  loss = criterion(outputs,targets)
  return loss 

In [ ]:
def train_fn(train_data_loader,model,optimizer,device,scheduler = None):
  model.train()
  losses = AverageMeter()
  rmse = AverageMeter()
  tk0 = tqdm(train_data_loader, total=len(train_data_loader))
  tot_loss = 0
  for bi,d in enumerate(tk0):
    images = d['image']
    labels = d['label']

    #send them to device 
    images = images.to(device,dtype=torch.float)
    labels = labels.to(device,dtype=torch.float)
    optimizer.zero_grad()
    if BOOL_CROP:
      bs, ncrops, c, h, w = images.size()
      outputs = model(images.view(-1, c, h, w)) # fuse batch size and ncrops
      outputs = outputs.view(bs, ncrops, -1).mean(1) # avg over crops
    
    else:
      outputs  = model(images)

    loss = loss_fn(outputs,labels.unsqueeze(1))
    loss.backward()
    optimizer.step()
    tot_loss = tot_loss + loss.item()

    #calculate rmse_score score for val 
    rmse.update(np.sqrt(loss.item()),labels.size(0))
    losses.update(loss.item(), labels.size(0))
    tk0.set_postfix(loss=losses.avg,rmse_score=rmse.avg)


    if scheduler is not None:
      scheduler.step()

  mse_score = tot_loss/len(train_data_loader)
  rmse_score = np.sqrt(mse_score)
  
  print("Training loss for this epoch: ", mse_score)
  print("Training rmse_score for this epoch: ", rmse_score)
  return rmse_score

In [ ]:
def eval_fn(valid_data_loader,model,device):
  model.eval()
  tot_loss = 0
  final_outputs = []
  final_targets = []
  with torch.no_grad():
    for bi,d in enumerate(valid_data_loader):

      images = d['image']
      labels = d['label']

      #send them to device 
      images = images.to(device,dtype=torch.float)
      labels = labels.to(device,dtype=torch.float)
      if BOOL_CROP:
        bs, ncrops, c, h, w = images.size()
        outputs = model(images.view(-1, c, h, w)) # fuse batch size and ncrops
        outputs = outputs.view(bs, ncrops, -1).mean(1) # avg over crops
      else:
        outputs  = model(images)

      loss = loss_fn(outputs,labels.unsqueeze(1))
      tot_loss = tot_loss + loss.item()

      
      final_outputs.append(outputs.cpu().detach().numpy())
      final_targets.append(labels.cpu().numpy())


    final_targets = np.concatenate(final_targets)
    final_outputs = np.concatenate(final_outputs)
    mse_score  = tot_loss/len(valid_data_loader)
    rmse_score = np.sqrt(mse_score)
    print("Validation loss for this epoch: ",mse_score)
    print('Validation rmse for this epoch',rmse_score)
  return rmse_score


In [ ]:
import albumentations as A
#from albumentations.pytorch.transforms import ToTensorV2
from albumentations.pytorch.transforms import ToTensor
from albumentations import torch as albtorch

input_shape = [128,256]

def get_transforms():
  train_transform = A.Compose([
                               
                              A.Resize(height=128, width=256,p=1),
                              #A.RandomBrightnessContrast(p=0.4),
                              A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), p=1.0),
                              albtorch.ToTensor(),
                               
                            ])
  val_transform = A.Compose([
                              A.Resize(height=128, width=256,p=1),
                              A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), p=1.0),
                              albtorch.ToTensor(),
                               
                            ])
  return train_transform,val_transform

## Stratfied Kfolds

In [ ]:
NAME = 'resnext101'
EPOCHS = 20
TRAIN_BATH_SIZE = 64
BOOL_CROP = False
skf = StratifiedKFold(n_splits=6,random_state=SEED_VAL)
all_rmse_scores = []
def run_folds():
  seed_all(SEED_VAL)
  train_transform,val_transform = get_transforms()
  train = pd.read_csv('Train.csv')
  train = train[train.label_quality == 2]
  train = train.rename(columns = {'UID':'image_name','growth_stage':'target'})
  DF = pd.DataFrame()
  for i,(train_index,val_index) in enumerate(skf.split(train,y=train.target)):
    print(f"#########################  Fold {i+1}/{skf.n_splits}  #########################")
    train_df , valid_df = train.iloc[train_index,:],train.iloc[val_index,:]
    train_df = train_df.reset_index()
    valid_df = valid_df.reset_index()
    train_dataset = ImageDataset('Images',train_df,train_transform)
    valid_dataset = ImageDataset('Images',valid_df,val_transform)
    train_data_loader = DataLoader(dataset=train_dataset,shuffle=True,batch_size=TRAIN_BATH_SIZE)
    valid_data_loader = DataLoader(dataset=valid_dataset,shuffle=False,batch_size=16)
    device = torch.device("cuda")
    model = Net(NAME)
    model.to(device)

    num_train_steps = int(len(train_df) /TRAIN_BATH_SIZE * EPOCHS )
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
    #scheduler = MultiStepLR(optimizer, milestones=[2,5,10], gamma=0.1)
    best_rmse = 1500
    
    for epoch in range(EPOCHS):
      print("----------------EPOCH "+str(epoch+1)+"---------------------")
      rmse_train = train_fn(train_data_loader, model, optimizer, device,scheduler=None)
      rmse_val = eval_fn(valid_data_loader ,model, device)
      if rmse_val<best_rmse:
        best_rmse = rmse_val 
        torch.save(model.state_dict(),f"best_model_{i}") 
    print(f'best VAL_RMSE for fold {i+1}: ',best_rmse)
    DF = DF.append({'FODL':1+i,'BEST_VAL_RMSE':best_rmse},ignore_index=True)
    all_rmse_scores.append(best_rmse)
  print(f"MEAN RMSE: {np.mean(all_rmse_scores)}")
  return DF


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [ ]:
kfold_results = run_folds()

#########################  Fold 1/6  #########################


Downloading: "https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth" to /root/.cache/torch/hub/checkpoints/resnext101_32x8d-8ba56ff5.pth



----------------EPOCH 1---------------------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


...


Training loss for this epoch:  0.05129995755851269
Training rmse_score for this epoch:  0.22649493936623108
Validation loss for this epoch:  0.25069445092231035
Validation rmse for this epoch 0.5006939693288809
----------------EPOCH 17---------------------



Training loss for this epoch:  0.03686938920107327
Training rmse_score for this epoch:  0.192014033864906
Validation loss for this epoch:  0.2302931003893415
Validation rmse for this epoch 0.4798886333195875
----------------EPOCH 18---------------------



Training loss for this epoch:  0.03274347304709648
Training rmse_score for this epoch:  0.18095157652559007
Validation loss for this epoch:  0.2746769556775689
Validation rmse for this epoch 0.5240963229002555
----------------EPOCH 19---------------------



Training loss for this epoch:  0.04853210892332228
Training rmse_score for this epoch:  0.22030004294897965
Validation loss for this epoch:  0.2680166924993197
Validation rmse for this epoch 0.5177032861585096
----------------EPOCH 20---------------------



Training loss for this epoch:  0.0552914419799651
Training rmse_score for this epoch:  0.23514132342054447
Validation loss for this epoch:  0.45601950734853747
Validation rmse for this epoch 0.675292164435911
best VAL_RMSE for fold 6:  0.4798886333195875
MEAN RMSE: 0.45079014183992766


In [ ]:
kfold_results


,BEST_VAL_RMSE,FODL
0,0.406946,1.0
1,0.432178,2.0
2,0.444374,3.0
3,0.471338,4.0
4,0.470015,5.0
5,0.479889,6.0


# Predicting

In [ ]:

test_transform = A.Compose([
                               
                              A.Resize(height=128, width=256,p=1),
                              
                              A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), p=1.0),
                              albtorch.ToTensor(),
                               
                            ])
test = pd.read_csv('SampleSubmission.csv')
test.rename(columns={'UID':'image_name'},inplace=True)
test_dataset = ImageDataset('Images',test,test_transform,mode='test')
test_data_loader = DataLoader(dataset=test_dataset,shuffle=False,batch_size=32)

## KFOLDS

In [ ]:
all_folds = []
for i in range(skf.n_splits):
  best_model = Net(NAME)
  best_model.load_state_dict(torch.load(f'best_model_{i}'))
  best_model.to(device)
  best_model.eval()
  final_outputs = []
  with torch.no_grad():
    tk0 = tqdm(test_data_loader, total=len(test_data_loader))
    for bi,d in enumerate(tk0):
      images = d['image']
      #send them to device 
      images = images.to(device,dtype=torch.float)
      if BOOL_CROP:
        bs, ncrops, c, h, w = images.size()
        outputs = best_model(images.view(-1, c, h, w)) # fuse batch size and ncrops
        outputs = outputs.view(bs, ncrops, -1).mean(1) # avg over crops
      else:
        outputs  = best_model(images)
      final_outputs.append(outputs.cpu().detach().numpy())
  final_outputs = np.concatenate(final_outputs)
  all_folds.append(final_outputs)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [ ]:
ss = pd.read_csv('/content/SampleSubmission.csv')

In [ ]:
ss['growth_stage'] = np.mean(all_folds,axis=0)
#ss['growth_stage'] = np.round(ss['growth_stage'])
ss['growth_stage'] = ss['growth_stage'].clip(1,7)
ss.head()

,UID,growth_stage
0,F3LbWkZq,4.552813
1,6P4wLVzI,3.999273
2,Tn6HIrdS,2.863723
3,EYM5R1TK,4.300880
4,fkcevlA0,5.242551


In [ ]:
ss.growth_stage.max(),ss.growth_stage.min()

(7.0, 1.8832578659057617)

In [ ]:
ss.to_csv(f'resnext101_Pytorch_regression_seed_20212020.csv',index=False)